
<a href="https://colab.research.google.com/github/takzen/ai-engineering-handbook/blob/main/notebooks/099_Neural_Architecture_Search_NAS.ipynb" target="_parent">
    <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>



<a href="https://colab.research.google.com/github/takzen/ai-engineering-handbook/blob/main/99_Neural_Architecture_Search_NAS.ipynb" target="_parent">
    <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>


# 🧬 NAS: Gdy AI projektuje AI (Neural Architecture Search)

Projektowanie sieci neuronowych to czarna magia. Dlaczego ResNet ma akurat tyle warstw? Bo ktoś tak sprawdził i zadziałało.

**NAS (Neural Architecture Search)** automatyzuje ten proces.
Traktujemy architekturę sieci jako **Genom**.
*   Genom: `[64, 128, 64]` -> Oznacza sieć z 3 warstwami ukrytymi o takich rozmiarach.
*   Genom: `[16]` -> Malutka sieć z 1 warstwą.

Zastosujemy **Ewolucję**:
1.  **Populacja:** Losujemy 10 różnych architektur.
2.  **Fitness:** Trenujemy każdą przez 1 epokę na MNIST. Wynik Accuracy to jej siła.
3.  **Mutacja:** Bierzemy najlepszą sieć i losowo zmieniamy liczbę neuronów lub dodajemy warstwę.

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import numpy as np
import copy
import random

# Konfiguracja
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
POPULATION_SIZE = 5  # Mała populacja dla szybkości
GENERATIONS = 3      # Krótka ewolucja
EPOCHS_PER_NET = 1   # Szybki test (tylko 1 epoka)

# Dane (MNIST)
train_loader = DataLoader(
    datasets.MNIST('data', train=True, download=True, transform=transforms.ToTensor()),
    batch_size=64, shuffle=True
)
test_loader = DataLoader(
    datasets.MNIST('data', train=False, transform=transforms.ToTensor()),
    batch_size=1000, shuffle=False
)

print(f"Urządzenie: {DEVICE}")

Urządzenie: cuda


## Dynamiczna Sieć (Budowniczy)

Musimy napisać klasę, która nie ma sztywnej struktury w `__init__`.
Zamiast tego przyjmuje listę `architecture` (np. `[100, 50]`) i dynamicznie buduje warstwy za pomocą `nn.ModuleList`.

In [2]:
class DynamicNet(nn.Module):
    def __init__(self, architecture):
        super().__init__()
        self.architecture = architecture
        self.layers = nn.ModuleList()
        
        # Wejście: 784 (MNIST 28x28)
        input_dim = 784
        
        # Budujemy warstwy ukryte na podstawie genotypu
        for hidden_dim in architecture:
            self.layers.append(nn.Linear(input_dim, hidden_dim))
            self.layers.append(nn.ReLU())
            input_dim = hidden_dim # Wyjście tej warstwy to wejście następnej
            
        # Warstwa wyjściowa (zawsze 10 klas)
        self.output_layer = nn.Linear(input_dim, 10)

    def forward(self, x):
        x = x.view(-1, 784) # Spłaszczamy obrazek
        for layer in self.layers:
            x = layer(x)
        return self.output_layer(x)

# Test: Stwórzmy losową sieć
dummy_net = DynamicNet([128, 64]) # 2 warstwy
print("Przykładowa architektura:")
print(dummy_net)

Przykładowa architektura:
DynamicNet(
  (layers): ModuleList(
    (0): Linear(in_features=784, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=64, bias=True)
    (3): ReLU()
  )
  (output_layer): Linear(in_features=64, out_features=10, bias=True)
)


## Funkcja Oceny (Fitness)

To funkcja, która:
1.  Bierze architekturę (listę).
2.  Buduje model.
3.  Trenuje go szybko (1 epoka).
4.  Zwraca Accuracy na zbiorze testowym.

To będzie nasz "koszt życia" dla danego osobnika.

In [3]:
def evaluate_architecture(arch):
    print(f"🧬 Testuję architekturę: {arch} ...", end=" ")
    
    # Budowa modelu
    model = DynamicNet(arch).to(DEVICE)
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss()
    
    # Szybki trening (1 epoka)
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(DEVICE), target.to(DEVICE)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        
        # Przerywamy po 100 batchach dla szybkości demo (opcjonalne)
        if batch_idx > 100: break 

    # Szybka ewaluacja
    model.eval()
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)
            pred = output.argmax(dim=1)
            correct += pred.eq(target).sum().item()
            
    acc = 100. * correct / len(test_loader.dataset)
    print(f"-> Accuracy: {acc:.2f}%")
    return acc

## Ewolucja (Search Loop)

1.  **Inicjalizacja:** Tworzymy losowe "mutanty" (np. `[30]`, `[100, 100]`).
2.  **Mutacja:**
    *   Zmień liczbę neuronów w warstwie (np. $50 \to 60$).
    *   Dodaj nową warstwę.
    *   Usuń warstwę.

Uruchamiamy proces na 3 pokolenia.

In [4]:
# 1. Populacja początkowa (losowe listy)
population = [
    [random.randint(32, 128) for _ in range(random.randint(1, 3))]
    for _ in range(POPULATION_SIZE)
]

def mutate(arch):
    """Dokonuje losowej zmiany w architekturze"""
    new_arch = arch.copy()
    choice = random.random()
    
    if choice < 0.33 and len(new_arch) > 1:
        # Usuń warstwę
        new_arch.pop(random.randint(0, len(new_arch)-1))
    elif choice < 0.66:
        # Zmień rozmiar warstwy
        idx = random.randint(0, len(new_arch)-1)
        new_arch[idx] = int(new_arch[idx] * random.uniform(0.5, 1.5))
    else:
        # Dodaj warstwę
        new_arch.insert(random.randint(0, len(new_arch)), random.randint(32, 128))
        
    # Zabezpieczenie przed pustą siecią
    if len(new_arch) == 0: return [32]
    return new_arch

# GŁÓWNA PĘTLA NAS
best_arch = None
best_acc = 0

for gen in range(GENERATIONS):
    print(f"\n--- POKOLENIE {gen+1} ---")
    scores = []
    
    # Ocena całej populacji
    for arch in population:
        acc = evaluate_architecture(arch)
        scores.append((acc, arch))
        
        if acc > best_acc:
            best_acc = acc
            best_arch = arch
    
    # Selekcja (Bierzemy 2 najlepsze)
    scores.sort(key=lambda x: x[0], reverse=True)
    survivors = [x[1] for x in scores[:2]]
    
    print(f"Najlepsi w tym pokoleniu: {survivors}")
    
    # Reprodukcja (Mutacja zwycięzców)
    new_population = list(survivors) # Elityzm (zostawiamy najlepszych)
    while len(new_population) < POPULATION_SIZE:
        parent = random.choice(survivors)
        child = mutate(parent)
        new_population.append(child)
        
    population = new_population

print("-" * 30)
print(f"🏆 ZWYCIĘZCA NAS: {best_arch}")
print(f"Wynik: {best_acc:.2f}%")


--- POKOLENIE 1 ---
🧬 Testuję architekturę: [38, 38, 87] ... -> Accuracy: 82.71%
🧬 Testuję architekturę: [103, 63, 39] ... -> Accuracy: 83.12%
🧬 Testuję architekturę: [115] ... -> Accuracy: 87.85%
🧬 Testuję architekturę: [58, 47, 62] ... -> Accuracy: 84.56%
🧬 Testuję architekturę: [111, 42, 52] ... -> Accuracy: 83.59%
Najlepsi w tym pokoleniu: [[115], [58, 47, 62]]

--- POKOLENIE 2 ---
🧬 Testuję architekturę: [115] ... -> Accuracy: 88.93%
🧬 Testuję architekturę: [58, 47, 62] ... -> Accuracy: 83.35%
🧬 Testuję architekturę: [58, 47] ... -> Accuracy: 87.13%
🧬 Testuję architekturę: [58, 47, 123, 62] ... -> Accuracy: 82.78%
🧬 Testuję architekturę: [154] ... -> Accuracy: 89.01%
Najlepsi w tym pokoleniu: [[154], [115]]

--- POKOLENIE 3 ---
🧬 Testuję architekturę: [154] ... -> Accuracy: 88.28%
🧬 Testuję architekturę: [115] ... -> Accuracy: 88.20%
🧬 Testuję architekturę: [38, 154] ... -> Accuracy: 87.34%
🧬 Testuję architekturę: [77] ... -> Accuracy: 87.56%
🧬 Testuję architekturę: [157] ... -> 

In [5]:
# Trenujemy zwycięzcę "na poważnie" (dłużej)
print("Trenowanie zwycięskiej architektury do pełna...")
final_model = DynamicNet(best_arch).to(DEVICE)
# ... tu byśmy zrobili normalny trening na 10 epok
print("Gotowe. AI zaprojektowało sobie mózg.")

Trenowanie zwycięskiej architektury do pełna...
Gotowe. AI zaprojektowało sobie mózg.


## 🧠 Podsumowanie: AutoML

To, co zrobiliśmy, to uproszczona wersja algorytmów takich jak **EfficientNet** (który został zaprojektowany przez Google AutoML, a nie człowieka).

**Zalety NAS:**
1.  Znajduje nieoczywiste architektury (np. bardzo wąskie, ale głębokie).
2.  Oszczędza czas inżyniera (ale kosztuje czas GPU).

**Wady:**
1.  Jest ekstremalnie kosztowny obliczeniowo (trzeba wytrenować setki modeli). Dlatego w praktyce używa się "One-Shot NAS" (współdzielenie wag między modelami), żeby nie trenować od zera.